# Writing a network and port scanner with Scapy
- replicate some basic features of Nmap and other network scanning tools

In [ ]:
!nmap

In [ ]:
# ping scan using nmap - ping scan mac host
! nmap -sn 192.168.65.1

## ping can be blocked
- disable ping on host and scan again

In [ ]:
! nmap -sn 192.168.65.1

In [ ]:
# enable os fingerprinting
! nmap -sS -A 192.168.65.1

## scapy ping scan
- scan the network with ping/icmp packets and scan for ports on the live hosts

In [ ]:
#! /usr/bin/python3

# Adapted for Python 3 from : https://thepacketgeek.com/scapy-p-10-emulating-nmap-functions/

from scapy.all import *
import netaddr
import random

def pingHost(host):
    resp = sr1(IP(dst=str(host))/ICMP(), timeout=2, verbose=0)
    up = False
    #resp.show()
    if not resp:
        print(str(host) + " is down or not responding.")
    elif resp.haslayer(ICMP):
        if (int(resp.getlayer(ICMP).type)==3 and int(resp.getlayer(ICMP).code) in [1,2,3,9,10,13]):
            print(str(host) + " is blocking ICMP.")
        elif (int(resp.getlayer(ICMP).type) == 0): # icmp-response received 
            up = True
        else:
            print("icmp type = {}".format(int(resp.getlayer(ICMP).type)))
            print(str(addr) + " is acting weird.")
    return up

# Send ICMP ping request, wait for answer
def pingScanNetwork(addresses):
    liveHosts = []
    for addr in addresses:
        if (addr == addresses.network or addr == addresses.broadcast):
            continue

        print("Trying on {}".format(addr))
        if pingHost(str(addr)):
            liveHosts.append(str(addr))
                
    return liveHosts

## scan a host

In [ ]:
host = 'www.facebook.com'
live = pingHost(host)
if live:
    print('{} host is up!'.format(host))

## scan a whole subnet

In [ ]:
# Define IP range to scan
network = "192.168.47.1/24"
# Define TCP port range
portRange = [22,23,80,443,449]
# make list of addresses out of network, set live host counter
addresses = netaddr.IPNetwork(network)
liveHosts = pingScanNetwork(addresses)

## scapy port scanner
- tcp syn scan to map open ports

In [ ]:
#! /usr/bin/python3
# Adapted from: https://thepacketgeek.com/scapy-p-10-emulating-nmap-functions/
# Fixed some bugs and ported for Python3

from scapy.all import *
import random
# Define end host and TCP port range

# Send SYN with random Src Port for each Dst port
def scanPorts(host, portRange):
    for dstPort in portRange:
        print("Trying port {:6}".format(dstPort))
        srcPort = random.randint(1025,65534)
        resp = sr1(IP(dst=host)/TCP(sport=srcPort,dport=dstPort,flags="S"),timeout=1,verbose=0)
        if resp:
            if (str(type(resp)) == "<type 'NoneType'>"):
                print(host + ":" + str(dstPort) + " is filtered (silently dropped).")
            elif(resp.haslayer(TCP)):
                if(resp.getlayer(TCP).flags == 0x12):
                    send_rst = sr(IP(dst=host)/TCP(sport=srcPort,dport=dstPort,flags="R"),timeout=1,verbose=0)
                    print (host + ":" + str(dstPort) + " is open.")
            elif (resp.getlayer(TCP).flags == 0x14):
                print (host + ":" + str(dstPort) + " is closed.")
            elif(resp.haslayer(ICMP)):
                if(int(resp.getlayer(ICMP).type)==3 and int(resp.getlayer(ICMP).code) in [1,2,3,9,10,13]):
                    print (host + ":" + str(dstPort) + " is filtered (silently dropped).")
        else:
            print("No response received.")
            

In [ ]:
host = "www.facebook.com"
portRange = [22,23,80,443,3389]
scanPorts(host, portRange)